In [1]:
# Import data

import pandas as pd
import os
import numpy as np

In [2]:
# Specify dataset directory

data_dir: str = '../dataset/CS2_36-38'
cleaned = '../dataset/cleaned'
df = pd.DataFrame()
hi = pd.DataFrame()

current = 'Current(A)'
voltage = 'Voltage(V)'
c_index = 'Cycle_Index'
min_voltage = 3.8
max_voltage = 4.2001

In [4]:
%pip install openpyxl
for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
        df_temp = df_temp[[c_index, current, voltage]]
        df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]
        df_temp = df_temp[df_temp[voltage].diff().gt(0)]

        grouped_df = df_temp.groupby(df_temp[c_index])

        for key, group in grouped_df:
            voltage_normalized = (group[voltage] - min_voltage) / (max_voltage - min_voltage)
            hi_v = np.trapz(voltage_normalized)
            hi_i = np.trapz(group[current])
            row = pd.DataFrame([{'hi_v': hi_v, 'hi_i': hi_i}])
            hi = pd.concat([hi, row])
            print(hi)

        # max_index = df_temp[c_index].max()
        # df_temp = [df.reset_index(drop=True) for _, df in df.groupby(df_temp[c_index])]
        # for index in df_temp:
        #     hi_v = np.trapz(index[voltage]) 
        #     hi_i = np.trapz(index[current]) 
        #     row = pd.DataFrame([{'hi_v': hi_v, 'hi_i':hi_i}])
        #     hi = pd.concat([hi, row])

        # df = pd.concat([df, df_temp[0]])

Note: you may need to restart the kernel to use updated packages.
         hi_v       hi_i
0   60.541765  53.064721
0   65.189405  58.817472
0   62.063610  56.717887
0   58.192896  53.087357
0   63.166517  56.719156
..        ...        ...
0   76.510665  86.644398
0   76.539807  86.644398
0   75.385781  86.093236
0   76.295361  86.091334
0   60.541765  53.064721

[2059 rows x 2 columns]
         hi_v       hi_i
0   60.541765  53.064721
0   65.189405  58.817472
0   62.063610  56.717887
0   58.192896  53.087357
0   63.166517  56.719156
..        ...        ...
0   76.539807  86.644398
0   75.385781  86.093236
0   76.295361  86.091334
0   60.541765  53.064721
0   65.189405  58.817472

[2060 rows x 2 columns]
         hi_v       hi_i
0   60.541765  53.064721
0   65.189405  58.817472
0   62.063610  56.717887
0   58.192896  53.087357
0   63.166517  56.719156
..        ...        ...
0   75.385781  86.093236
0   76.295361  86.091334
0   60.541765  53.064721
0   65.189405  58.817472
0   62.06

In [5]:
for key, value in grouped_df:
    print(value) 
# hi.to_csv(f'{cleaned}/hi.csv')

     Cycle_Index  Current(A)  Voltage(V)
10             1    0.550153    3.805480
11             1    0.550153    3.808557
12             1    0.550153    3.811635
13             1    0.550153    3.814388
14             1    0.550153    3.817466
..           ...         ...         ...
176            1    0.550153    4.191784
177            1    0.550153    4.195185
178            1    0.549789    4.198911
179            1    0.550153    4.200045
187            1    0.871066    4.200045

[171 rows x 3 columns]
     Cycle_Index  Current(A)  Voltage(V)
332            2    0.550153    3.805642
333            2    0.550153    3.808881
334            2    0.549971    3.812282
335            2    0.550153    3.816008
336            2    0.550153    3.819247
..           ...         ...         ...
491            2    0.549789    4.189517
492            2    0.549789    4.193242
493            2    0.550153    4.197129
494            2    0.550335    4.200045
510            2    0.472108    4

In [6]:
import os
# Create the directory if it doesn't exist
os.makedirs(cleaned, exist_ok=True)

# Save the file
df.to_csv(f'{cleaned}/cleaned.csv')
